In [1]:
import pandas as pd

In [6]:
postcodes_geojson_paths = pd.read_json("./data/postcodes_to_paths.json")

Join bin days

In [19]:
bin_days = (
    pd.read_json("./data/bin_days.json", orient="index")
    .reset_index()
    .rename(columns={"index": "postcode", 0: "bin_day"})
    .dropna()
)

print(bin_days.shape)

bin_days.head()

(1564, 2)


,postcode,bin_day
0,HP18 9UH,Wednesday
1,HP18 9UY,Wednesday
2,HP18 9UZ,Wednesday
3,HP18 9XA,Wednesday
4,HP18 9XB,Wednesday


In [20]:
merged = bin_days.merge(postcodes_geojson_paths, on="postcode")
merged

,postcode,bin_day,geojson_path
0,HP18 9UH,Wednesday,./data/geojson/sectors/HP18/HP18 9.geojson
1,HP18 9UY,Wednesday,./data/geojson/sectors/HP18/HP18 9.geojson
2,HP18 9UZ,Wednesday,./data/geojson/sectors/HP18/HP18 9.geojson
3,HP18 9XA,Wednesday,./data/geojson/sectors/HP18/HP18 9.geojson
4,HP18 9XB,Wednesday,./data/geojson/sectors/HP18/HP18 9.geojson
...,...,...,...
1559,OX16 5NU,Tuesday,./data/geojson/sectors/OX16/OX16 5.geojson
1560,OX16 5NX,Tuesday,./data/geojson/sectors/OX16/OX16 5.geojson
1561,OX16 5PY,Monday,./data/geojson/sectors/OX16/OX16 5.geojson
1562,OX16 5QF,Wednesday,./data/geojson/sectors/OX16/OX16 5.geojson


Get bin days as dummy columns?

In [46]:
bin_days_by_sector = (
    pd.get_dummies(merged, columns=["bin_day"])
    .groupby("geojson_path")[
        [
            "bin_day_Friday",
            "bin_day_Monday",
            "bin_day_Thursday",
            "bin_day_Tuesday",
            "bin_day_Wednesday",
        ]
    ]
    .max()
    .assign(bin_days=lambda x: x.sum(axis=1))
    .reset_index()
    .assign(
        sector=lambda x: x["geojson_path"].str.split("/").str[-1].str.split(".").str[0]
    )
)

bin_days_by_sector

,geojson_path,bin_day_Friday,bin_day_Monday,bin_day_Thursday,bin_day_Tuesday,bin_day_Wednesday,bin_days,sector
0,./data/geojson/sectors/HP18/HP18 9.geojson,False,False,False,False,True,1,HP18 9
1,./data/geojson/sectors/MK18/MK18 4.geojson,False,False,False,True,False,1,MK18 4
2,./data/geojson/sectors/NN13/NN13 5.geojson,False,False,False,True,False,1,NN13 5
3,./data/geojson/sectors/OX15/OX15 0.geojson,False,False,True,True,False,2,OX15 0
4,./data/geojson/sectors/OX15/OX15 4.geojson,False,True,True,True,True,4,OX15 4
5,./data/geojson/sectors/OX15/OX15 5.geojson,False,False,True,True,True,3,OX15 5
6,./data/geojson/sectors/OX15/OX15 6.geojson,True,False,False,True,False,2,OX15 6
7,./data/geojson/sectors/OX16/OX16 0.geojson,True,True,True,True,True,5,OX16 0
8,./data/geojson/sectors/OX16/OX16 1.geojson,True,False,True,True,True,4,OX16 1
9,./data/geojson/sectors/OX16/OX16 2.geojson,True,False,False,True,True,3,OX16 2


In [47]:
bin_days_by_sector.to_csv("./data/bin_days_by_sector.csv", index=False)